In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
DIR_PATH = '/kaggle/input/cat-and-dog'
TRAIN_DIR_PATH = os.path.join(DIR_PATH, "training_set", "training_set")
TEST_DIR_PATH = os.path.join(DIR_PATH, "test_set", "test_set")
print(TRAIN_DIR_PATH)
# for dirname, _, filenames in os.walk('/kaggle/input/cat-and-dog'):    
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cat-and-dog/training_set/training_set


In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2025-11-19 09:41:14.836268: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763545274.874863     229 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763545274.883396     229 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [3]:
IMG_SIZE = (128, 128)
BATCH = 32

In [4]:
train = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True, 
    zoom_range = 0.1
).flow_from_directory(
    TRAIN_DIR_PATH,
    target_size = IMG_SIZE,
    batch_size = BATCH,
    class_mode = 'binary', 
    shuffle = True
)

print("Class indices", train.class_indices)

Found 8005 images belonging to 2 classes.
Class indices {'cats': 0, 'dogs': 1}


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer = Adam(1e-1), loss = 'binary_crossentropy', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-11-19 09:41:22.468897: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,769 (12.61 MB)

 Trainable params: 3,304,769 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
EPOCHS = 6
history = model.fit(train, epochs = EPOCHS)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/6
251/251 ━━━━━━━━━━━━━━━━━━━━ 149s 584ms/step - accuracy: 0.5075 - loss: 1312.5511
Epoch 2/6
251/251 ━━━━━━━━━━━━━━━━━━━━ 149s 591ms/step - accuracy: 0.5080 - loss: 0.6953
Epoch 3/6
251/251 ━━━━━━━━━━━━━━━━━━━━ 146s 579ms/step - accuracy: 0.4965 - loss: 0.6969
Epoch 4/6
251/251 ━━━━━━━━━━━━━━━━━━━━ 143s 570ms/step - accuracy: 0.5052 - loss: 0.6972
Epoch 5/6
251/251 ━━━━━━━━━━━━━━━━━━━━ 144s 573ms/step - accuracy: 0.4913 - loss: 0.7027
Epoch 6/6
251/251 ━━━━━━━━━━━━━━━━━━━━ 141s 563ms/step - accuracy: 0.4927 - loss: 0.6966


In [14]:
# TEST_DIR  = os.path.join(BASE_DIR, "test_set", "test_set")
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

test_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    TEST_DIR_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode='binary',
    shuffle=False
)

Found 2023 images belonging to 2 classes.


In [15]:
test_gen.reset()
y_prob = model.predict(test_gen).ravel()
y_pred = (y_prob >= 0.5).astype(int)
y_true = test_gen.classes

acc  = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred)
rec  = recall_score(y_true, y_pred)
f1   = f1_score(y_true, y_pred)

print(f"Accuracy : {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall   : {rec:.4f}")
print(f"F1-score : {f1:.4f}")

64/64 ━━━━━━━━━━━━━━━━━━━━ 11s 170ms/step
Accuracy : 0.5002
Precision: 0.5002
Recall   : 1.0000
F1-score : 0.6669


In [17]:
model.save_weights('my_model.weights.h5')

In [18]:
metrics_data = {
    "model_name": "ClassificationModel",
    "timestamp": "2025-11-19T15:35:00Z",
    "dataset_used": "Dog and Cat",
    "metrics": {
        "accuracy": acc,
        "f1_score": f1,
        "precision" : prec,
        "Recall" : rec
    }
}

In [19]:
metrics_data

{'model_name': 'ClassificationModel',
 'timestamp': '2025-11-19T15:35:00Z',
 'dataset_used': 'Dog and Cat',
 'metrics': {'accuracy': 0.5002471576866041,
  'f1_score': 0.6668863261943987,
  'precision': 0.5002471576866041,
  'Recall': 1.0}}

In [20]:
import json

file_path = "metrics_v1.json"
with open(file_path, 'w') as f:
    json.dump(metrics_data, f, indent=4) # indent for pretty-printing